## This code containts forecasts using time series methods (ES and AR) and ML methods (tree ensembles using GB and XGB) for Calls Data

In [ ]:
# Commonly used python functions and display settings
import pandas as pd
import numpy as np
pd.options.display.float_format = '{:,.2f}'.format

from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"
from IPython.core.display import display, HTML

import warnings
warnings.filterwarnings("ignore") # specify to ignore warning messages

In [ ]:
# Key imports for this code (various ML and Stat Models)
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.ensemble import RandomForestRegressor
from xgboost import XGBRegressor
from sklearn.linear_model import LinearRegression
from statsmodels.tsa.api import ExponentialSmoothing
from statsmodels.tsa.statespace.sarimax import SARIMAX
import statsmodels.api as sm
from statsmodels.tsa.stattools import acf
from statsmodels.tsa.stattools import pacf
import pmdarima as pm
from pmdarima import model_selection
from pmdarima import auto_arima

In [ ]:
# import viz libraries
import matplotlib.pyplot as plt
import plotly
plotly.offline.init_notebook_mode(connected=True)
from plotly.graph_objs import *
from plotly import tools
import plotly.graph_objects as go
from matplotlib import pyplot
from pandas.plotting import autocorrelation_plot

### Preliminary Data Analysis

In [ ]:
# fetch data from the CSV file
calls_df = pd.read_csv('calls_data.csv', parse_dates = ['date'])

calls_df.head()
calls_df.tail()

In [ ]:
# Creating a plot of calls made over time
plot_data = []
plot_data.append(go.Scatter(x= calls_df['date'], y= calls_df['calls']))
layout = go.Layout(xaxis = dict(title='Date'), yaxis = dict(title= 'Calls Made'), 
                   title = 'Time Series of Monthly Calls Made')
fig = go.Figure(data= plot_data, layout=layout)

plotly.offline.iplot(fig)

In [ ]:
# Zooming into the 100th to 119th data points of above plot
plot_data = []
plot_data.append(go.Scatter(x= calls_df['date'][100:120], y= calls_df['calls'][100:120]))
layout = go.Layout(xaxis = dict(title='Date'), yaxis = dict(title= 'Calls Made'), 
                   title = 'Time Series of Monthly Calls Made')
fig = go.Figure(data= plot_data, layout=layout)

plotly.offline.iplot(fig)

In [ ]:
# Representing above plot using dots
plot_data = []
plot_data.append(go.Scatter(x= calls_df['date'][100:120], y= calls_df['calls'][100:120], mode = 'markers'))
layout = go.Layout(xaxis = dict(title='Date'), yaxis = dict(title= 'Calls Made'), 
                   title = 'Time Series of Monthly Calls Made')
fig = go.Figure(data= plot_data, layout=layout)

plotly.offline.iplot(fig)

In [ ]:
# This creates a graph of the autocorrelation function versus lags for the calls data
sm.graphics.tsa.plot_acf(calls_df['calls'].values.squeeze(), lags=40)

In [ ]:
# This creates a graph of the partial autocorrelation function versus lags for the calls data
sm.graphics.tsa.plot_pacf(calls_df['calls'].values.squeeze(), lags=40)

In [ ]:
# subset data to only include 300 data points (most recent)
# ES and ARIMA methods are better with < 200 training data points

subset_data = calls_df[calls_df['date'] > '1994-01-01'].reset_index(drop = True)
len(subset_data)
subset_data.head()
subset_data.isna().sum()

## Exponential Smoothing

In [ ]:
# First we get the time series
time_series = subset_data['calls']

# Define the seasonal periods
seasonal_periods = 12 

# Define the number of predictions to make 
h = 1

# Define the length of each training set
T = 200

# Initialize the lists to store the percentage and absolute errors
perc_error_list = []
abs_error_list = []

es_preds_train = np.zeros(T+h) # In case we wish to use the ES predictions

# Loop through the data frame and make predictions using exponential smoothing
for i in range(len(time_series) - T - h):
    # Define the training and testing data sets
    train = time_series.iloc[i:i+T].values
    test = time_series.iloc[i+T:i+T+h].values
    
    # Fit the exponential smoothing model
    model = ExponentialSmoothing(train, trend='add', seasonal='add', seasonal_periods=seasonal_periods)
    fit_model = model.fit()
    
    # Make predictions
    pred_list = fit_model.forecast(h)
    preds = pred_list[h-1]

    # Calculate percentage and absolute errors
    perc_errors = np.abs(test[h-1]-preds)/test[h-1]
    abs_errors = np.abs(test[h-1]-preds)

    # Store the percentage and absolute errors
    perc_error_list.append(perc_errors)
    abs_error_list.append(abs_errors)
    
    # Get the ES predictions
    es_preds_train = np.append(es_preds_train, preds) 

In [ ]:
# Print the percentage-error results
print('Mean absolute percentage error:', np.mean(perc_error_list, axis = 0))
print('Median absolute percentage error:', np.median(perc_error_list, axis = 0))
print('75th percentile of absolute percentage error:', np.percentile(perc_error_list, 75, axis = 0))
print('90th percentile of absolute percentage error:', np.percentile(perc_error_list, 90, axis = 0))

In [ ]:
# Print the absolute error ratio results
avg_global = subset_data['calls'][T+h:].mean()
print('Mean absolute error ratio:', np.mean(abs_error_list, axis = 0)/avg_global)
print('Median absolute error ratio:', np.median(abs_error_list, axis = 0)/avg_global)
print('75th percentile absolute error ratio:', np.percentile(abs_error_list, 75, axis = 0)/avg_global)
print('90th percentile absolute error ratio:', np.percentile(abs_error_list, 90, axis = 0)/avg_global)

## SARIMAX initial model building and then train & predict

In [ ]:
# Getting optimal differencing 
d_opt = pm.arima.ndiffs(subset_data['calls'].iloc[0:T])
d_opt
D_opt = pm.arima.nsdiffs(subset_data['calls'].iloc[0:T], m = seasonal_periods)
D_opt

In [ ]:
# Graphical study of the effect of differencing
subset_data['diff1'] = subset_data['calls'] - subset_data['calls'].shift(periods = 1)
subset_data['diff2'] = subset_data['diff1'] - subset_data['diff1'].shift(periods = 1)
subset_data.head()

sm.graphics.tsa.plot_acf(subset_data['calls'].values.squeeze(), lags=40)
sm.graphics.tsa.plot_acf(subset_data['diff1'].dropna().values.squeeze(), lags=40)
sm.graphics.tsa.plot_acf(subset_data['diff2'].dropna().values.squeeze(), lags=40)

In [ ]:
# That we just to show, now we need to remove the columns we created in the previous cell
subset_data.drop(columns = ['diff1', 'diff2'], inplace = True)
subset_data.head()

In [ ]:
# Initialize the lists to store the percentage and absolute errors
ar_perc_error_list = []
ar_abs_error_list = []

ar_preds_train = np.zeros(T+h) # In case we wish to use the ARIMA predictions

# Loop through the data frame and make predictions using ARIMA
for i in range(len(time_series) - T - h):
    # Define the training and testing data sets
    train = time_series.iloc[i:i+T].values
    test = time_series.iloc[i+T:i+T+h].values

    # Using a specified order (this would need to be fine-tuned)
    order = (1, 1, 0) 
    seasonal_order = (1, 0, 0, seasonal_periods) 

    # Fit the SARIMAX or ARIMA model
    model = SARIMAX(endog=train, exog=None, order=order, seasonal_order=seasonal_order)
    fit_model = model.fit(disp=False)

    # Make predictions
    pred_list = fit_model.forecast(steps=len(test), exog=None)
    preds = pred_list[h-1]

    # Calculate percentage and absolute errors
    ar_perc_errors = np.abs(test[h-1]-preds)/test[h-1]
    ar_abs_errors = np.abs(test[h-1]-preds)

    # Store the percentage and absolute errors
    ar_perc_error_list.append(ar_perc_errors)
    ar_abs_error_list.append(ar_abs_errors)
    
    # Get the ARIMA predictions
    ar_preds_train = np.append(ar_preds_train, preds) 

In [ ]:
# Print the percentage-error results
print('Mean absolute percentage error:', np.mean(ar_perc_error_list, axis = 0))
print('Median absolute percentage error:', np.median(ar_perc_error_list, axis = 0))
print('75th percentile of absolute percentage error:', np.percentile(ar_perc_error_list, 75, axis = 0))
print('90th percentile of absolute percentage error:', np.percentile(ar_perc_error_list, 90, axis = 0))

In [ ]:
# Print the absolute error ratio results
print('Mean absolute error ratio:', np.mean(ar_abs_error_list, axis = 0)/avg_global)
print('Median absolute error ratio:', np.median(ar_abs_error_list, axis = 0)/avg_global)
print('75th percentile absolute error ratio:', np.percentile(ar_abs_error_list, 75, axis = 0)/avg_global)
print('90th percentile absolute error ratio:', np.percentile(ar_abs_error_list, 90, axis = 0)/avg_global)

### For Gradient Boosting we first do some trend adjustment

In [ ]:
# For the ML Models we use the dates to obtain the month and year as features
subset_data['month'] = subset_data['date'].dt.month
subset_data['year'] = subset_data['date'].dt.year
subset_data.head()
subset_data.tail()

In [ ]:
# To get the trend, we average across all months of a year (using only training data)
yearly_avg = subset_data[['calls', 'year']].iloc[0:T].groupby(['year']).mean()
yearly_avg

In [ ]:
# We can bring the year back as a feature (above it is in the index, so we reset it)
yearly_avg.reset_index(inplace = True)

In [ ]:
# Regression model to fit a linear trend model

model = LinearRegression(fit_intercept = True)
model.fit(yearly_avg['year'].array.reshape(-1, 1), yearly_avg['calls']) # When extending to multiple features remove .array.reshape(-1, 1)

# The following gives the R-square score
model.score(yearly_avg['year'].array.reshape(-1, 1), yearly_avg['calls'])

# This is the coefficient Beta_1 (or slope of the Simple Linear Regression line)
model.coef_

# This is the coefficient Beta_0
model.intercept_

In [ ]:
# Applying the trend for the test and train dataset across all months
subset_data['trend'] = -2817.955 + 1.4995*(subset_data['year']+subset_data['month']/12)
subset_data.head()

In [ ]:
# Plotting the trend line (regression line) along with the monthly calls data
plot_data = []
plot_data.append(go.Scatter(x= subset_data['date'], y= subset_data['calls'], name = 'calls'))
plot_data.append(go.Scatter(x= subset_data['date'], y= subset_data['trend'], name = 'trend'))
layout = go.Layout(xaxis = dict(title='Date'), yaxis = dict(title= 'Calls Made and Trend'), 
                   title = 'Time Series of Monthly Calls Made with Trend')
fig = go.Figure(data= plot_data, layout=layout)

plotly.offline.iplot(fig)

In [ ]:
# To get some of the lag features later we would need the full data set, and extract the date and year; trend
calls_df['month'] = calls_df['date'].dt.month
calls_df['year'] = calls_df['date'].dt.year
calls_df['trend'] = -2817.955 + 1.4995*(calls_df['year']+calls_df['month']/12)

In [ ]:
# Residuals (de-trending) and shifting them to get lags by 1 month and 12 months
subset_data['resid_lags'] = (calls_df['calls']-calls_df['trend']).shift(periods = 
                                                seasonal_periods)[-len(subset_data):].reset_index(drop = True)
subset_data['resid_lag1'] = (calls_df['calls']-calls_df['trend']).shift(periods = 
                                                        1)[-len(subset_data):].reset_index(drop = True)
subset_data.head(15)
subset_data.tail()

In [ ]:
# Obtaining the training data (note: unlike ES and ARIMA, here we do not retrain)
X_train = subset_data[0:T]
X_train['residual'] = X_train['calls'] - X_train['trend']
X_train.head()
X_train.tail()
# The "y" variable that we predict is the residual itself
y_train = X_train['residual']
y_train

In [ ]:
# The remaining 100 data points are test data
X_test = subset_data[T:]
X_test.head()
X_test.tail()

## GB (Gradient Boosting)

In [ ]:
# Just so we have the test and training data for the GB to choose the right features in the next cell
X_train
X_test

In [ ]:
# One shot training (i.e., no re-training)

# defining the model and parameters
gb = GradientBoostingRegressor(n_estimators = 100, max_depth = 6, min_samples_leaf = 2)

# Asking the model to fit the training data (features used: month, resid_lags, resid_lag1)
gb = gb.fit(X_train.drop(columns = ['date', 'calls', 'year', 'trend', 'residual']), y_train) 

# Asking what the importance of features
gb.feature_importances_

In [ ]:
# Testing for one shot training
# Initialize the lists to store the percentage and absolute errors
gb_perc_error_list = []
gb_abs_error_list = []

# Make predictions using Gradient Boosting with a single run of train

y_test = X_test['calls'] # This is the final prediction that we will compare against

# Make predictions (residuals plus trend)
y_preds = gb.predict(X_test.drop(columns = ['date', 'calls', 'year', 'trend'])) + X_test['trend']

# Calculate percentage and absolute errors
perc_errors = np.abs(y_test-y_preds)/y_test
abs_errors = np.abs(y_test-y_preds)

# Store the percentage and absolute errors
gb_perc_error_list.append(perc_errors)
gb_abs_error_list.append(abs_errors)

In [ ]:
# Print the percentage-error results
print('Mean absolute percentage error:', np.mean(gb_perc_error_list))
print('Median absolute percentage error:', np.median(gb_perc_error_list))
print('75th percentile of absolute percentage error:', np.percentile(gb_perc_error_list, 75))
print('90th percentile of absolute percentage error:', np.percentile(gb_perc_error_list, 90))

In [ ]:
# Print the absolute error ratio results
print('Mean absolute error ratio:', np.mean(gb_abs_error_list)/avg_global)
print('Median absolute error ratio:', np.median(gb_abs_error_list)/avg_global)
print('75th percentile absolute error ratio:', np.percentile(gb_abs_error_list, 75)/avg_global)
print('90th percentile absolute error ratio:', np.percentile(gb_abs_error_list, 90)/avg_global)

In [ ]:
# Comparing the forecasts against the actuals, graphically
plot_data = []
plot_data.append(go.Scatter(x= subset_data['date'][T+1:], y= subset_data['calls'][T+1:], name = 'calls'))
plot_data.append(go.Scatter(x= subset_data['date'][T+1:], y= y_preds[1:], name = 'GB', mode = 'markers'))
plot_data.append(go.Scatter(x= subset_data['date'][T+1:], y= es_preds_train[T+1:], name = 'ES', mode = 'markers'))
plot_data.append(go.Scatter(x= subset_data['date'][T+1:], y= ar_preds_train[T+1:], name = 'AR', mode = 'markers'))
layout = go.Layout(xaxis = dict(title='Date'), yaxis = dict(title= 'Calls made versus predicted'), 
                   title = 'Monthly Calls Test Data vs. Predictions')
fig = go.Figure(data= plot_data, layout=layout)

plotly.offline.iplot(fig)

In [ ]:
# Model forecasts versus actuals across 2 years
plot_data = []
plot_data.append(go.Scatter(x= subset_data['date'][T+1:T+25], y= subset_data['calls'][T+1:T+25], name = 'calls'))
plot_data.append(go.Scatter(x= subset_data['date'][T+1:T+25], y= y_preds[1:T+25], name = 'GB', mode = 'markers'))
plot_data.append(go.Scatter(x= subset_data['date'][T+1:T+25], y= es_preds_train[T+1:T+25], name = 'ES', mode = 'markers'))
plot_data.append(go.Scatter(x= subset_data['date'][T+1:T+25], y= ar_preds_train[T+1:T+25], name = 'AR', mode = 'markers'))
layout = go.Layout(xaxis = dict(title='Date'), yaxis = dict(title= 'Calls made versus predicted'), 
                   title = 'Monthly Calls Test Data vs. Predictions')
fig = go.Figure(data= plot_data, layout=layout)

plotly.offline.iplot(fig)

## XGBoost

In [ ]:
# If we simply use the parameters without hyper parameter tuning
# One shot training

# Define the XGBoost regressor with specific hyperparameters
model = XGBRegressor(
    n_estimators=100,
    max_depth=6,
    learning_rate=0.1,
    subsample=1.0,
    colsample_bytree=1.0,
    objective='reg:squarederror',
    random_state=42
)

# Asking the model to fit the training data
model.fit(X_train.drop(columns = ['date', 'calls', 'year', 'trend', 'residual']), y_train) 

In [ ]:
# One shot training
# Initialize the lists to store the percentage and absolute errors
xgb_perc_error_list = []
xgb_abs_error_list = []

# Make predictions using Xtreme Gradient Boosting with a single run of train

y_test = X_test['calls']

# Make predictions
y_preds = model.predict(X_test.drop(columns = ['date', 'calls', 'year', 'trend'])) + X_test['trend']

# Calculate percentage and absolute errors
perc_errors = np.abs(y_test-y_preds)/y_test
abs_errors = np.abs(y_test-y_preds)

# Store the percentage and absolute errors
xgb_perc_error_list.append(perc_errors)
xgb_abs_error_list.append(abs_errors)

In [ ]:
# Print the percentage-error results
print('Mean absolute percentage error:', np.mean(xgb_perc_error_list, axis = 1))
print('Median absolute percentage error:', np.median(xgb_perc_error_list, axis = 1))
print('75th percentile of absolute percentage error:', np.percentile(xgb_perc_error_list, 75, axis = 1))
print('90th percentile of absolute percentage error:', np.percentile(xgb_perc_error_list, 90, axis = 1))

In [ ]:
# Print the absolute error ratio results
print('Mean absolute error ratio:', np.mean(xgb_abs_error_list, axis = 1)/avg_global)
print('Median absolute error ratio:', np.median(xgb_abs_error_list, axis = 1)/avg_global)
print('75th percentile absolute error ratio:', np.percentile(xgb_abs_error_list, 75, axis = 1)/avg_global)
print('90th percentile absolute error ratio:', np.percentile(xgb_abs_error_list, 90, axis = 1)/avg_global)